# Abstract

DETR3D (Deformable Transformer) refers to a nolve approach for r multi-camera 3D object detection. The architecture extracts 2D features from multiple camera images and then uses a sparse set of 3D object queries to index into these 2D features, linking 3D positions to multi-view images using camera transformation matrices.

DETR3D is considered an advancement over the previous standard methods of depth Prediction such as FCOS3D(Fully Convolutional One-Stage Monocular Object Detection) and  [Objects as Points](https://arxiv.org/abs/1904.07850). These methods use their specified way to estimate the depth of a particular RGB point in an image.

DETR3D proposes a more graceful transition between 2D observations and 3D predictions for autonomous driving, which does not rely on a module for dense depth prediction. The framework, DETR3D (Multi-View 3D Detection), addresses this problem in a top-down fashion. It links 2D feature extraction and 3D object prediction via geometric back-projection with camera transformation matrices. The method starts from a sparse set of object priors, shared across the dataset and learned end-to-end.

# Overview

To get a brief understanding of DETR 3D, we can paint a braod picture here and then get into the details of each part.

- A backbone(Resnet + FPN) is given a set of multi view input images

- The backbone from the given input encodes these images and gives out a set of Sparse Object Queries wherein each query represents a 3D reference point.

- 2D features are then transformed to refine the object queries by projecting the 3D reference point into the image space. This is done via the detection head which links the computed 2D features to a set of 3D bounding box predictions in a geometry-aware manner. Each layer of the detection head starts from a sparse set
of object queries, which are learned from the data. Each object query encodes a 3D location, which
is projected to the camera planes and used to collect image features via bilinear interpolation.



## Feature Learning

This step is straighforward as they use a backbone such as resnet to extract bounding boxes from the images provided. These images are also labelled with the ground truth which contains the bounding boxes, angles, velocity etc. manually labelled to provide an accurate representation fo the moving objects.

A Feature Pyramid network is also used to enhance these said features, the images are divided into 4 feature sets *$(F_1, F_2, F_3, F_4)$* where in each
$F_k = (f_1, f_2 ... f_6)$ here each `f` corresponds to a feature in the 6 camera angels provided.



## Detection Head

This is the most important part of this work. They have devised a way where they can simultaneously compare 2D features with 3D bounding boxes to get an estiamte of which 2D bounding box fits the 3D bounding cuboid best.

As quoted in the paper, traditionally the way of finding 2D bounding boxes is done with the following.

*`Existing methods for detecting objects from camera input typically employ a bottom-up approach,
which predicts a dense set of bounding boxes per image, filters redundant boxes between the images,
and aggregates predictions across cameras in a post-processing step. This paradigm has two crucial
drawbacks: dense bounding box prediction requires accurate depth perception, which itself is a
challenging problem; and NMS-based redundancy removal and aggregation are non-parallelizable
operations that introduce significant inference overhead.`*

DETR3D uses a top down approach, where a few steps are repeated in layer of the model.

1. Calculate centers of 2D bounding boxes.
2. Project these centers into all feature maps.
3. sample features via bilinear interpolation and incorporate them into object queries
4. Use multi head attention to get desired results.

